## Script to convert xyz to pdb using the [atomic simulation environment](https://ase-lib.org/index.html)
Install with: `pip install --upgrade ase`

In [1]:
import numpy as np
import glob
from ase.io import read, write

In [44]:
# read

for filexyz in glob.glob('structures/*.xyz'):
    print (filexyz)
    atoms = read(filexyz)

    # select not H
    non_h_indices = [atom.index for atom in atoms if atom.symbol != 'H']
    atoms = atoms[non_h_indices]

    # find coordinates for TYR and PHE
    if 'Tyr' in filexyz:
        num_aro = 8 
        name_aro = ['TYR']
    elif 'Phe' in filexyz:
        num_aro = 7
        name_aro = ['PHE']

    atomsARO = atoms[:num_aro]
    dist_matrix = atomsARO.get_all_distances()

    cutoff = 1.7
    adj_matrix = (dist_matrix <= cutoff) & (dist_matrix > 0)
    coordination_numbers = np.sum(adj_matrix, axis=1)

    atom_labels = []
    #print (coordination_numbers)
    ir = 0
    for i,a in enumerate(atomsARO):
        #print (i,a, coordination_numbers[i])
        if coordination_numbers[i] >= 2:
            ir+=1
            atom_labels.append('CR%i'%ir)
        elif coordination_numbers[i] == 1 and a.symbol == 'O':
            atom_labels.append('OH')
        elif coordination_numbers[i] == 1 and a.symbol == 'C':
            atom_labels.append('CB')

    atom_labels = np.array(atom_labels, dtype='U4')
    res_names = np.array(name_aro*num_aro, dtype='U3')
    res_numbers = np.array([1]*num_aro, dtype=int)

    atomsARO.set_array('atomtypes', atom_labels)
    atomsARO.set_array('residuenumbers', res_numbers)
    atomsARO.set_array('residuenames', res_names, str)

    # find coordinates for ARG and LYS
    num_chr = atoms.get_number_of_atoms() - atomsARO.get_number_of_atoms()
    if "Arg" in filexyz:
        name_chr = ['ARG']
    elif "Lys" in filexyz:
        name_chr = ['LYS']

    atomsCHR = atoms[num_aro:]
    atom_labels = []
    for i, symbol in enumerate(atomsCHR.get_chemical_symbols()):
        label = f"{symbol}{i+1}"
        atom_labels.append(label)

    atom_labels = np.array(atom_labels, dtype='U4')
    res_names = np.array(name_chr*num_chr, dtype='U3')
    res_numbers = np.array([2]*num_chr, dtype=int)

    atomsCHR.set_array('atomtypes', atom_labels)
    atomsCHR.set_array('residuenumbers', res_numbers)
    atomsCHR.set_array('residuenames', res_names, str)

    ## combine atoms
    atoms = atomsARO +  atomsCHR
    
    fileout = filexyz.replace('.xyz', '.pdb')
    print (f" Writing to PDB file: {fileout}\n")
    write(fileout, atoms)
    #

structures/ArgTyrT.xyz
 Writing to PDB file: structures/ArgTyrT.pdb

structures/LysPhe.xyz
 Writing to PDB file: structures/LysPhe.pdb

structures/ArgTyrP.xyz
 Writing to PDB file: structures/ArgTyrP.pdb

structures/ArgPhe.xyz
 Writing to PDB file: structures/ArgPhe.pdb

structures/ArgTyr.xyz
 Writing to PDB file: structures/ArgTyr.pdb

structures/LysTyr.xyz
 Writing to PDB file: structures/LysTyr.pdb

structures/ArgPheT.xyz
 Writing to PDB file: structures/ArgPheT.pdb



/tmp/ipykernel_428766/3317105900.py:48: FutureWarning: Please use len(self) or, if your atoms are distributed, self.get_global_number_of_atoms.
  num_chr = atoms.get_number_of_atoms() - atomsARO.get_number_of_atoms()
